In [ ]:
import requests
import json
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
# 调用 Hugging Face API 的函数
def your_api_call_function(input_text, model_id, hf_token, prompt_template):
    api_url = "https://api-inference.huggingface.co/models/" + model_id
    headers = {
        "Authorization": f"Bearer {hf_token}",
        "Content-Type": "application/json"
    }
    payload = {"inputs": input_text, "parameters": {"max_length": 50}}
    prompt = prompt_template.format(input=input_text)
    payload["inputs"] = prompt
    response = requests.post(api_url, headers=headers, data=json.dumps(payload))
    if response.status_code == 200:
        response_data = response.json()
        return response_data[0]["generated_text"]
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return ""

# 读取数据的函数
def read_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data['inputs'], data['references'], data['truelabels']

In [ ]:

# 主程序
def main(file_path, threshold1,threshold2, model_id, hf_token):
    inputs, references, truelabels = read_data(file_path)
    
    predicted_labels = []
    for input_text, reference, truelabel in zip(inputs, references, truelabels):
        # 构造prompt
        prompt_template = "Continue the sentence based on the given input. Calculate the ROUGE score between the generated result and the reference. If the score is greater than {threshold1}, label it as infringement. If the score is lower than {threshold2}, label it as non-infringement. Finally, directly output the label indicating whether it is infringing or not."
        prompt = prompt_template.format(threshold1=threshold1,threshold2=threshold2)
        
        # 调用大模型API
        label = your_api_call_function(input_text, model_id, hf_token, prompt)
        
        predicted_labels.append(label)
        
        print(f"Input: {input_text}")
        print(f"Reference: {reference}")
        print(f"Label: {label}, True Label: {truelabel}\n")
    
    f1 = f1_score(truelabels, predicted_labels, average='macro')
    acc = accuracy_score(truelabels, predicted_labels)
    
    return f1, acc

In [ ]:
# 设置阈值
threshold1 = 0.7
threshold2 = 0.7

# 文件路径
file_path = 'path_to_your_data_file.json'

# Hugging Face 模型 ID 和 API 令牌
model_id = "YOUR_MODEL_ID"  # 替换为你的模型 ID
hf_token = "YOUR_HF_TOKEN"  # 替换为你的 Hugging Face API 令牌

# 运行程序
f1, acc = main(file_path, threshold1, threshold2, model_id, hf_token)
print(f"F1 Score: {f1}")
print(f"Accuracy: {acc}")